In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn import linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 50)
from sklearn.pipeline import Pipeline
from os import listdir
from os.path import isfile, join

In [87]:
data_folder = "C:/Users/Byron/Documents/Football Predictions/UnderstatPredictions/Datasets"

In [90]:
TeamNameLookup = pd.read_csv(data_folder+"/TeamNameLookup.csv")
TeamNameLookup.head()

,league,FootballData,Understat
0,Bundesliga,Augsburg,Augsburg
1,Bundesliga,Bayern Munich,Bayern Munich
2,Bundesliga,Darmstadt,Darmstadt
3,Bundesliga,Dortmund,Borussia Dortmund
4,Bundesliga,Ein Frankfurt,Eintracht Frankfurt


In [84]:
seasons={"1415":"2014/15","1516":"2015/16","1617":"2016/17","1718":"2017/18","1819":"2018/19","1920":"2019/20"}
#seasons={"1415":"2014/15"}
leagues = {"E0":"EPL","SP1":"La Liga","F1":"Ligue 1","I1":"Serie A","D1":"Bundesliga"}

for i in seasons.keys():
    print(i)

1415
1516
1617
1718
1819
1920


In [179]:
#seasons = [1415,1516,1617,1718,1819,1920]
sample_df = pd.read_csv("http://www.football-data.co.uk/mmz4281/1920/E0.csv")
main_cols= ["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR"]
bookie_dict = {"Bet365":["B365H","B365D","B365A"],"BetWin":["BWH","BWD","BWA"],"Average":["AvgH","AvgD","AvgA"]
               ,"BetVictor":["VCH","VCD","VCA"],"WilliamHill":["WHH","WHD","WHA"]}

bookie_cols = [item for sublist in bookie_dict.values() for item in sublist]
print(bookie_cols)

def get_fd_data(seasons,leagues):
    '''
    Pulls data from Football Data website for seasons and leagues as defined
    Seasons: Dictionary format. Item should be the value required for Football-Data url. Value should be your naming convention
    leagues: Dictionary format. Item should be the value required for Football-Data url. Value should be your naming convnetion.
    '''
    fd_all = pd.DataFrame()
    for i in seasons.keys():
        for j in leagues.keys():
            url = "http://www.football-data.co.uk/mmz4281/" + i +"/" + j + ".csv"
            print(url)
            one_year = pd.read_csv(url)
            if j != "2019/20":
                #print(one_year.columns)
                #one_year.drop(columns = ["AVH","AVD","AVA"])
                one_year = one_year.rename(columns = {"BbAvH":"AvgH"
                                          ,"BbAvD":"AvgD"
                                          ,"BbAvA":"AvgA"})
            one_year = one_year[main_cols+bookie_cols]
            one_year["season"] = seasons[i]
            one_year["league"] = leagues[j]
            fd_all=fd_all.append(one_year)
    return fd_all

df_fd = get_fd_data(seasons,leagues)



['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'AvgH', 'AvgD', 'AvgA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA']
http://www.football-data.co.uk/mmz4281/1415/E0.csv
http://www.football-data.co.uk/mmz4281/1415/SP1.csv
http://www.football-data.co.uk/mmz4281/1415/F1.csv
http://www.football-data.co.uk/mmz4281/1415/I1.csv
http://www.football-data.co.uk/mmz4281/1415/D1.csv
http://www.football-data.co.uk/mmz4281/1516/E0.csv
http://www.football-data.co.uk/mmz4281/1516/SP1.csv
http://www.football-data.co.uk/mmz4281/1516/F1.csv
http://www.football-data.co.uk/mmz4281/1516/I1.csv
http://www.football-data.co.uk/mmz4281/1516/D1.csv
http://www.football-data.co.uk/mmz4281/1617/E0.csv
http://www.football-data.co.uk/mmz4281/1617/SP1.csv
http://www.football-data.co.uk/mmz4281/1617/F1.csv
http://www.football-data.co.uk/mmz4281/1617/I1.csv
http://www.football-data.co.uk/mmz4281/1617/D1.csv
http://www.football-data.co.uk/mmz4281/1718/E0.csv
http://www.football-data.co.uk/mmz4281/1718/SP1.csv
http://www.foo

In [154]:
#test = df_fd.copy()
df_fd = test.copy()

In [180]:
df_fd = pd.merge(df_fd,TeamNameLookup, left_on=["league","HomeTeam"], 
                                         right_on=["league","FootballData"]).drop(columns ="FootballData")
df_fd = df_fd.rename(columns={"Understat":"US_hometeam"})
df_fd = pd.merge(df_fd,TeamNameLookup, left_on=["league","AwayTeam"],
                                        right_on=["league","FootballData"]).drop(columns="FootballData")
df_fd = df_fd.rename(columns={"Understat":"US_awayteam"})
df_fd["fixture"] = df_fd["US_hometeam"] + " vs " + df_fd["US_awayteam"]
df_fd.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,B365H,B365D,B365A,BWH,BWD,BWA,AvgH,AvgD,AvgA,VCH,VCD,VCA,WHH,WHD,WHA,season,league,US_hometeam,US_awayteam,fixture
0,E0,16/08/14,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,1.25,5.5,12.00,1.25,5.96,12.43,1.25,6.25,10.5,1.25,5.50,12.0,2014/15,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
1,E0,17/04/16,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,1.33,5.0,8.75,1.34,5.20,9.06,1.33,5.50,10.0,1.35,4.80,9.0,2015/16,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
2,E0,01/01/17,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,1.30,5.5,9.75,1.33,5.45,9.57,1.33,5.50,10.5,1.33,5.50,9.0,2016/17,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
3,E0,20/01/2018,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,1.50,4.4,6.25,1.50,4.46,6.55,1.50,4.60,7.0,1.50,4.33,7.0,2017/18,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
4,E0,21/04/2019,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,1.53,4.5,5.75,1.54,4.44,6.03,1.53,4.40,6.5,1.52,4.33,6.5,2018/19,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace


In [186]:
main_cols = ["league","season","Date","fixture","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR"]
main_cols

['league',
 'season',
 'Date',
 'fixture',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR']

In [156]:
print(df_fd.groupby("league")["B365A"].describe(include = "all"))

             count      mean       std   min     25%  50%     75%   max
league                                                                 
Bundesliga  1736.0  4.556411  4.163744  1.14  2.4500  3.3  5.0000  34.0
EPL         2170.0  5.068161  4.544801  1.12  2.3800  3.5  5.7500  41.0
La Liga     2149.0  5.369981  5.457960  1.08  2.5000  3.6  5.5000  41.0
Ligue 1     2159.0  4.546512  3.648039  1.13  2.7000  3.6  5.0000  34.0
Serie A     2246.0  4.793085  3.879356  1.16  2.2925  3.4  5.9375  34.0


Get rid of nulls in the dataframe

In [158]:
print(len(df_fd))
df_fd.dropna(inplace =True)
print(len(df_fd))

10461
10451


In [159]:
df_fd.to_csv(data_folder+"/football_data_results.csv")

In [167]:
for i in bookie_dict.items():
    print(i)

('Bet365', ['B365H', 'B365D', 'B365A'])
('BetWin', ['BWH', 'BWD', 'BWA'])
('Average', ['AvgH', 'AvgD', 'AvgA'])
('BetVictor', ['VCH', 'VCD', 'VCA'])
('WilliamHill', ['WHH', 'WHD', 'WHA'])


In [207]:
def bookie_format(df,bookie_dict):
    '''
    Renames your chosen bookie odds into a standardised Home Draw Away odds format with the bookie name as an extra variable
    df: DataFrame for you to apply the renaming to
    bookie_dict: Dictionary of bookie names and relevant bookie columns
    '''
    final_data = df.copy()
    bookie_name = i[0]
    bookie_cols = i[1]

    final_cols = main_cols+bookie_cols
    final_data = final_data[final_cols]
    final_data['bookie'] = bookie_name
    final_cols = main_cols + ["BookieHomeOdds","BookieDrawOdds","BookieAwayOdds","bookie",]
    print(final_cols)
    final_data.columns = final_cols
    return final_data

bookie_output = pd.DataFrame()
for i in bookie_dict.items():
    one_bookie = bookie_format(df_fd,i)
    bookie_output = bookie_output.append(one_bookie)

bookie_output.head()

['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']
['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']
['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']
['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']
['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']


,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie
0,EPL,2014/15,16/08/14,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,Bet365
1,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,Bet365
2,EPL,2016/17,01/01/17,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,Bet365
3,EPL,2017/18,20/01/2018,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,Bet365
4,EPL,2018/19,21/04/2019,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,Bet365


In [208]:
def convert_odds_to_prob(odds_col):
    bookie_prob = 1/odds_col
    return bookie_prob

def calculate_bookie_margin(row):
    total = row["BookieHomeProb"] + row["BookieDrawProb"] + row["BookieAwayProb"]
    margin = (total - 1)*100
    return margin


def most_likely_result(Home,Draw,Away):
    mx = np.array([Home,Draw,Away]).max()
    #print(mx)
    if np.isnan(mx) ==False:
        if Home == mx:
            result =  "H"
        elif Draw == mx:
            result = "D"
        else:
            result =  "A"
        return [result,mx]

def final_result_prob(row):
    if row["FTR"]=="H":
        return row["BookieHomeProb"]
    elif row["FTR"] =="D":
        return row['BookieDrawProb']
    else:
        return row["BookieAwayProb"]

bookie_output['BookieHomeProb'] = bookie_output['BookieHomeOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output['BookieDrawProb'] = bookie_output['BookieDrawOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output['BookieAwayProb'] = bookie_output['BookieAwayOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output["BookieMargin"] = bookie_output.apply(lambda row: calculate_bookie_margin(row),axis =1)

bookie_output[["BookiePredResult","BookiePredResultProb"]] = bookie_output.apply(lambda row: 
                                                                 pd.Series(most_likely_result(row["BookieHomeProb"]
                                                                                              ,row["BookieDrawProb"]
                                                                                              ,row["BookieAwayProb"])),axis =1)
bookie_output["BookieFinalResultProb"] = bookie_output.apply(lambda row: final_result_prob(row),axis=1)

    

In [209]:
bookie_output["BookiePredCorrect"] = np.where(bookie_output["FTR"]==bookie_output["BookiePredResult"],1,0)
bookie_output.head()

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect
0,EPL,2014/15,16/08/14,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,Bet365,0.800000,0.153846,0.066667,2.051282,H,0.800000,0.800000,1
1,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,Bet365,0.751880,0.173913,0.105263,3.105590,H,0.751880,0.173913,0
2,EPL,2016/17,01/01/17,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,Bet365,0.775194,0.181818,0.105263,6.227514,H,0.775194,0.775194,1
3,EPL,2017/18,20/01/2018,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,Bet365,0.666667,0.210526,0.142857,2.005013,H,0.666667,0.666667,1
4,EPL,2018/19,21/04/2019,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,Bet365,0.653595,0.217391,0.153846,2.483223,H,0.653595,0.153846,0


In [210]:
bookie_output.groupby(["league","season","bookie"])["BookiePredCorrect","BookieFinalResultProb"].mean().reset_index().head(60)

,league,season,bookie,BookiePredCorrect,BookieFinalResultProb
0,Bundesliga,2014/15,Average,0.519608,0.428250
1,Bundesliga,2014/15,Bet365,0.513072,0.429489
2,Bundesliga,2014/15,BetVictor,0.516340,0.420554
3,Bundesliga,2014/15,BetWin,0.516340,0.429206
4,Bundesliga,2014/15,WilliamHill,0.522876,0.429137
5,Bundesliga,2015/16,Average,0.526144,0.441524
6,Bundesliga,2015/16,Bet365,0.532680,0.441163
7,Bundesliga,2015/16,BetVictor,0.522876,0.436588
8,Bundesliga,2015/16,BetWin,0.522876,0.442184
9,Bundesliga,2015/16,WilliamHill,0.529412,0.440895


In [211]:
bookie_output.to_csv(data_folder+"/bookie_odds.csv")